In [1]:
import numpy as np
import os
import cv2 as cv

In [2]:
dirs = os.listdir('..')
images_dir = list(filter(lambda directory: directory.startswith('images'), dirs))[0]
n_files = len(os.listdir(f'../{images_dir}'))

In [6]:
def slam():
    orb = cv.ORB_create()
    bf = cv.BFMatcher(cv.NORM_HAMMING)

    img1 = cv.imread(f'../{images_dir}/0.jpg', cv.IMREAD_GRAYSCALE)
    pre_points, pre_features = orb.detectAndCompute(img1, None)

    curr_frame_index = 1
    first_frame = img1

    map_initialised = False

    res = []

    for k in range(1):
        curr_frame = cv.imread(f'../{images_dir}/{curr_frame_index}.jpg', cv.IMREAD_GRAYSCALE)
        curr_points, curr_features = orb.detectAndCompute(curr_frame, None)

        curr_frame_index += 1

        matches = bf.knnMatch(pre_features, curr_features, k=2)

        pre_matched_points = []
        curr_matched_points = []
        # ratio test as per Lowe's paper
        for i, (m, n) in enumerate(matches):
            if m.distance < 0.8 * n.distance:
                curr_matched_points.append(curr_points[m.trainIdx].pt)
                pre_matched_points.append(pre_points[m.queryIdx].pt)

        pre_matched_points = np.array(pre_matched_points)
        curr_matched_points = np.array(curr_matched_points)
        # tform, inlier_tform_idx = cv.findHomography(pre_matched_points, curr_matched_points, method=cv.RANSAC)

        F, mask = cv.findFundamentalMat(pre_matched_points, curr_matched_points, method=cv.FM_LMEDS)

        inlier_pre_points = pre_matched_points[mask.ravel() == 1]
        inlier_curr_points = curr_matched_points[mask.ravel() == 1]

        E, _ = cv.findEssentialMat(inlier_pre_points, inlier_curr_points)

        _, R, t, _ = cv.recoverPose(E, inlier_pre_points, inlier_curr_points)

        K = np.matrix(
            [[692.1783869, 0., 353.07150929], [0., 704.4016042, 178.98211959], [0., 0., 1.]]).A
        dist = np.array([[0.12111298, 0.26876559, -0.04746641, -0.00175447, -1.04781593]])

        pre_pts_norm = cv.undistortPoints(inlier_pre_points, K, dist)
        curr_pts_norm = cv.undistortPoints(inlier_curr_points, K, dist)

        P1 = np.hstack((K, np.zeros((3, 1))))
        P2 = np.hstack((np.dot(K, R), np.dot(K, t)))

        points_4d_homogeneous = cv.triangulatePoints(P1, P2, pre_pts_norm, curr_pts_norm)

        points_3d = cv.convertPointsFromHomogeneous(points_4d_homogeneous.T)

        camera_location = -np.dot(R.T, t)
        
        return (points_3d[:, 0, 0], points_3d[:, 0, 1], points_3d[:, 0, 2])
        # ax.scatter3D(camera_location[0], camera_location[1], camera_location[2], c=camera_location[2], cmap='Blues')

        # map_initialised = True

xpoints, ypoints, z = slam()

array([618.48922274, 616.03839617, 619.63857665, 615.99524185,
       618.76168516, 618.79873004, 621.23904845, 621.25233326,
       618.58588935, 619.47256756, 617.79284582, 618.47824413,
       619.32776646, 617.99641847, 621.55094929, 619.77351031,
       619.33550672, 618.62261743, 619.03925285, 619.52765251,
       618.35914197, 617.91364464, 616.35145146, 619.40557393,
       619.32838355, 618.82138063, 618.3150906 , 618.54195236,
       619.81727461, 618.616278  , 619.08069087, 617.46128603,
       617.3631913 , 618.38251654, 618.17762724, 619.92753142,
       615.46688547, 618.72533527, 618.28018835, 619.80173577,
       618.56180061, 619.33097879, 619.56944466, 618.53168457,
       618.49125347, 618.76253223, 619.34797262, 619.36142632,
       617.95469734, 620.98400172, 619.60949667, 620.60979771,
       615.33263705, 618.28726115, 623.12540349, 619.53470721,
       619.49296118, 620.00655929, 618.6512392 , 618.36225563,
       619.70417362, 619.59686002, 619.66382803, 618.99

In [9]:
# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=x,  # <-- Put your data instead
    y=[4, 5, 6],  # <-- Put your data instead
    z=[7, 8, 9],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)